In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
import numpy as np

import Audio_functions as af
from functools import partial
import datetime

import librosa
import soundfile as sf
import IPython.display as ipd

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
data_folder = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/backgrounds'
#data_folder = 'C:/Users/Toby/Speech_Data/LibriSpeech/train-other-500'
sample_rate = 16384
n_parallel_readers = 4
n_channels = 1

In [ ]:
data_pipe = tf.data.Dataset.list_files(data_folder + '/*.wav').map(partial(af.read_audio,
                                                                            sample_rate=sample_rate,
                                                                            n_channels=n_channels), 
                                                                    num_parallel_calls=n_parallel_readers)

In [ ]:
sess = tf.Session()
next_file = data_pipe.make_one_shot_iterator().get_next()

In [2]:
length = 0
file_count = 0
max_file_length = 0
min_file_length = 100000

while True:
    try:
        file = sess.run(next_file)
        len_file = len(file) / sample_rate
        length += len_file
        if len_file > max_file_length:
            max_file_length = len_file
        if len_file < min_file_length:
            min_file_length = len_file
        file_count += 1
        if file_count % 10 == 0:
            print('{fc} files processed'.format(fc=file_count))
    except tf.errors.OutOfRangeError:
        mean_file_length = length / file_count
        print('Files in folder: {fc}'.format(fc=file_count))
        print('Total length of audio in folder = {l}s ({lh} hours)'.format(l=round(length, 2), 
                                                                           lh=str(datetime.timedelta(seconds=length))))
        print('Maximum file length = {maxfl}s'.format(maxfl=round(max_file_length, 2)))
        print('Minimum file length = {minfl}s'.format(minfl=round(min_file_length, 2)))
        print('Mean file length = {meanfl}s'.format(meanfl=round(mean_file_length, 2)))
        break

NameError: name 'sess' is not defined

In [4]:
model_config = {'saving': True,  # Whether to take checkpoints
                'loading': False,  # Whether to load an existing checkpoint
                'dataset': 'LibriSpeech',  # Choice of 'LibriSpeech', 'CHiME', or 'both'
                'local_run': False,  # Whether experiment is running on laptop or server
                'checkpoint_to_load': "26/26-20",  # Checkpoint format: run/run-epoch
                'INITIALISATION_TEST': False,  # Whether or not to calculate test metrics before training
                'SAMPLE_RATE': 8192,  # Desired sample rate of audio. Input will be resampled to this
                'N_FFT': 512,  # Number of samples in each fourier transform
                'FFT_HOP': 128,  # Number of samples between the start of each fourier transform
                'N_PARALLEL_READERS': 2,
                'PATCH_WINDOW': 256,
                'PATCH_HOP': 128,
                'BATCH_SIZE': 5,
                'N_SHUFFLE': 20,
                'EPOCHS': 1,  # Number of full passes through the dataset to train for
                'EARLY_STOPPING': True,  # Should validation data checks be used for early stopping?
                'VAL_BY_EPOCHS': True,  # Validation at end of each epoch or every 'val_iters'?
                'VAL_ITERS': 2000,  # Number of training iterations between validation checks,
                'NUM_WORSE_VAL_CHECKS': 3,  # Number of successively worse validation checks before early stopping,
                'NORMALISE_MAG': True
                }

if model_config['local_run']:  # Data and Checkpoint directories on my laptop
    model_config['data_root'] = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/'
    model_config['model_base_dir'] = 'C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints'
    model_config['log_dir'] = 'logs/local'

else:  # Data and Checkpoint directories on the uni server
    model_config['chime_data_root'] = '/data/CHiME3/data/audio/16kHz/isolated/'
    model_config['librispeech_data_root'] = 'C:/Users/Toby/Speech_Data/LibriSpeech/'
    model_config['model_base_dir'] = '/home/enterprise.internal.city.ac.uk/acvn728/checkpoints'
    model_config['log_dir'] = 'logs/ssh'

In [5]:
import Dataset
train, val, test = Dataset.prepare_datasets(model_config)

In [6]:
next_item = train.make_one_shot_iterator().get_next()

In [7]:
voice_audio = next_item[3]
sess = tf.Session()

In [8]:
test = sess.run(next_item)

In [11]:
test[0].shape

(5, 256, 257, 2)

In [ ]:
iter_count = 0
while True:
    try:
        voice_wave = sess.run(voice_audio)
        iter_count += 1
        print('iter_count: {}'.format(iter_count))
    except tf.errors.OutOfRangeError:
        print('Finished. iter_count: {}'.format(iter_count))
        break

    

iter_count: 1
iter_count: 2
iter_count: 3
iter_count: 4
iter_count: 5
iter_count: 6
iter_count: 7
iter_count: 8
iter_count: 9
iter_count: 10
iter_count: 11
iter_count: 12
iter_count: 13
iter_count: 14
iter_count: 15
iter_count: 16
iter_count: 17
iter_count: 18
iter_count: 19
iter_count: 20
iter_count: 21
iter_count: 22
iter_count: 23
iter_count: 24
iter_count: 25
iter_count: 26
iter_count: 27
iter_count: 28
iter_count: 29
iter_count: 30
iter_count: 31
iter_count: 32
iter_count: 33
iter_count: 34
iter_count: 35
iter_count: 36
iter_count: 37
iter_count: 38
iter_count: 39
iter_count: 40
iter_count: 41
iter_count: 42
iter_count: 43
iter_count: 44
iter_count: 45
iter_count: 46
iter_count: 47
iter_count: 48
iter_count: 49
iter_count: 50
iter_count: 51
iter_count: 52
iter_count: 53
iter_count: 54
iter_count: 55
iter_count: 56
iter_count: 57
iter_count: 58
iter_count: 59
iter_count: 60
iter_count: 61
iter_count: 62
iter_count: 63
iter_count: 64
iter_count: 65
iter_count: 66
iter_count: 67
iter

iter_count: 520
iter_count: 521
iter_count: 522
iter_count: 523
iter_count: 524
iter_count: 525
iter_count: 526
iter_count: 527
iter_count: 528
iter_count: 529
iter_count: 530
iter_count: 531
iter_count: 532
iter_count: 533
iter_count: 534
iter_count: 535
iter_count: 536
iter_count: 537
iter_count: 538
iter_count: 539
iter_count: 540
iter_count: 541
iter_count: 542
iter_count: 543
iter_count: 544
iter_count: 545
iter_count: 546
iter_count: 547
iter_count: 548
iter_count: 549
iter_count: 550
iter_count: 551
iter_count: 552
iter_count: 553
iter_count: 554
iter_count: 555
iter_count: 556
iter_count: 557
iter_count: 558
iter_count: 559
iter_count: 560
iter_count: 561
iter_count: 562
iter_count: 563
iter_count: 564
iter_count: 565
iter_count: 566
iter_count: 567
iter_count: 568
iter_count: 569
iter_count: 570
iter_count: 571
iter_count: 572
iter_count: 573
iter_count: 574
iter_count: 575
iter_count: 576
iter_count: 577
iter_count: 578
iter_count: 579
iter_count: 580
iter_count: 581
iter_cou

iter_count: 1031
iter_count: 1032
iter_count: 1033
iter_count: 1034
iter_count: 1035
iter_count: 1036
iter_count: 1037
iter_count: 1038
iter_count: 1039
iter_count: 1040
iter_count: 1041
iter_count: 1042
iter_count: 1043
iter_count: 1044
iter_count: 1045
iter_count: 1046
iter_count: 1047
iter_count: 1048
iter_count: 1049
iter_count: 1050
iter_count: 1051
iter_count: 1052
iter_count: 1053
iter_count: 1054
iter_count: 1055
iter_count: 1056
iter_count: 1057
iter_count: 1058
iter_count: 1059
iter_count: 1060
iter_count: 1061
iter_count: 1062
iter_count: 1063
iter_count: 1064
iter_count: 1065
iter_count: 1066
iter_count: 1067
iter_count: 1068
iter_count: 1069
iter_count: 1070
iter_count: 1071
iter_count: 1072
iter_count: 1073
iter_count: 1074
iter_count: 1075
iter_count: 1076
iter_count: 1077
iter_count: 1078
iter_count: 1079
iter_count: 1080
iter_count: 1081
iter_count: 1082
iter_count: 1083
iter_count: 1084
iter_count: 1085
iter_count: 1086
iter_count: 1087
iter_count: 1088
iter_count: 10

iter_count: 1514
iter_count: 1515
iter_count: 1516
iter_count: 1517
iter_count: 1518
iter_count: 1519
iter_count: 1520
iter_count: 1521
iter_count: 1522
iter_count: 1523
iter_count: 1524
iter_count: 1525
iter_count: 1526
iter_count: 1527
iter_count: 1528
iter_count: 1529
iter_count: 1530
iter_count: 1531
iter_count: 1532
iter_count: 1533
iter_count: 1534
iter_count: 1535
iter_count: 1536
iter_count: 1537
iter_count: 1538
iter_count: 1539
iter_count: 1540
iter_count: 1541
iter_count: 1542
iter_count: 1543
iter_count: 1544
iter_count: 1545
iter_count: 1546
iter_count: 1547
iter_count: 1548
iter_count: 1549
iter_count: 1550
iter_count: 1551
iter_count: 1552
iter_count: 1553
iter_count: 1554
iter_count: 1555
iter_count: 1556
iter_count: 1557
iter_count: 1558
iter_count: 1559
iter_count: 1560
iter_count: 1561
iter_count: 1562
iter_count: 1563
iter_count: 1564
iter_count: 1565
iter_count: 1566
iter_count: 1567
iter_count: 1568
iter_count: 1569
iter_count: 1570
iter_count: 1571
iter_count: 15

iter_count: 1996
iter_count: 1997
iter_count: 1998
iter_count: 1999
iter_count: 2000
iter_count: 2001
iter_count: 2002
iter_count: 2003
iter_count: 2004
iter_count: 2005
iter_count: 2006
iter_count: 2007
iter_count: 2008
iter_count: 2009
iter_count: 2010
iter_count: 2011
iter_count: 2012
iter_count: 2013
iter_count: 2014
iter_count: 2015
iter_count: 2016
iter_count: 2017
iter_count: 2018
iter_count: 2019
iter_count: 2020
iter_count: 2021
iter_count: 2022
iter_count: 2023
iter_count: 2024
iter_count: 2025
iter_count: 2026
iter_count: 2027
iter_count: 2028
iter_count: 2029
iter_count: 2030
iter_count: 2031
iter_count: 2032
iter_count: 2033
iter_count: 2034
iter_count: 2035
iter_count: 2036
iter_count: 2037
iter_count: 2038
iter_count: 2039
iter_count: 2040
iter_count: 2041
iter_count: 2042
iter_count: 2043
iter_count: 2044
iter_count: 2045
iter_count: 2046
iter_count: 2047
iter_count: 2048
iter_count: 2049
iter_count: 2050
iter_count: 2051
iter_count: 2052
iter_count: 2053
iter_count: 20

...

iter_count: 19788
iter_count: 19789
iter_count: 19790
iter_count: 19791
iter_count: 19792
iter_count: 19793
iter_count: 19794
iter_count: 19795
iter_count: 19796
iter_count: 19797
iter_count: 19798
iter_count: 19799
iter_count: 19800
iter_count: 19801
iter_count: 19802
iter_count: 19803
iter_count: 19804
iter_count: 19805
iter_count: 19806
iter_count: 19807
iter_count: 19808
iter_count: 19809
iter_count: 19810
iter_count: 19811
iter_count: 19812
iter_count: 19813
iter_count: 19814
iter_count: 19815
iter_count: 19816
iter_count: 19817
iter_count: 19818
iter_count: 19819
iter_count: 19820
iter_count: 19821
iter_count: 19822
iter_count: 19823
iter_count: 19824
iter_count: 19825
iter_count: 19826
iter_count: 19827
iter_count: 19828
iter_count: 19829
iter_count: 19830
iter_count: 19831
iter_count: 19832
iter_count: 19833
iter_count: 19834
iter_count: 19835
iter_count: 19836
iter_count: 19837
iter_count: 19838
iter_count: 19839
iter_count: 19840
iter_count: 19841
iter_count: 19842
iter_count

iter_count: 20246
iter_count: 20247
iter_count: 20248
iter_count: 20249
iter_count: 20250
iter_count: 20251
iter_count: 20252
iter_count: 20253
iter_count: 20254
iter_count: 20255
iter_count: 20256
iter_count: 20257
iter_count: 20258
iter_count: 20259
iter_count: 20260
iter_count: 20261
iter_count: 20262
iter_count: 20263
iter_count: 20264
iter_count: 20265
iter_count: 20266
iter_count: 20267
iter_count: 20268
iter_count: 20269
iter_count: 20270
iter_count: 20271
iter_count: 20272
iter_count: 20273
iter_count: 20274
iter_count: 20275
iter_count: 20276
iter_count: 20277
iter_count: 20278
iter_count: 20279
iter_count: 20280
iter_count: 20281
iter_count: 20282
iter_count: 20283
iter_count: 20284
iter_count: 20285
iter_count: 20286
iter_count: 20287
iter_count: 20288
iter_count: 20289
iter_count: 20290
iter_count: 20291
iter_count: 20292
iter_count: 20293
iter_count: 20294
iter_count: 20295
iter_count: 20296
iter_count: 20297
iter_count: 20298
iter_count: 20299
iter_count: 20300
iter_count

iter_count: 20702
iter_count: 20703
iter_count: 20704
iter_count: 20705
iter_count: 20706
iter_count: 20707
iter_count: 20708
iter_count: 20709
iter_count: 20710
iter_count: 20711
iter_count: 20712
iter_count: 20713
iter_count: 20714
iter_count: 20715
iter_count: 20716
iter_count: 20717
iter_count: 20718
iter_count: 20719
iter_count: 20720
iter_count: 20721
iter_count: 20722
iter_count: 20723
iter_count: 20724
iter_count: 20725
iter_count: 20726
iter_count: 20727
iter_count: 20728
iter_count: 20729
iter_count: 20730
iter_count: 20731
iter_count: 20732
iter_count: 20733
iter_count: 20734
iter_count: 20735
iter_count: 20736
iter_count: 20737
iter_count: 20738
iter_count: 20739
iter_count: 20740
iter_count: 20741
iter_count: 20742
iter_count: 20743
iter_count: 20744
iter_count: 20745
iter_count: 20746
iter_count: 20747
iter_count: 20748
iter_count: 20749
iter_count: 20750
iter_count: 20751
iter_count: 20752
iter_count: 20753
iter_count: 20754
iter_count: 20755
iter_count: 20756
iter_count

iter_count: 21158
iter_count: 21159
iter_count: 21160
iter_count: 21161
iter_count: 21162
iter_count: 21163
iter_count: 21164
iter_count: 21165
iter_count: 21166
iter_count: 21167
iter_count: 21168
iter_count: 21169
iter_count: 21170
iter_count: 21171
iter_count: 21172
iter_count: 21173
iter_count: 21174
iter_count: 21175
iter_count: 21176
iter_count: 21177
iter_count: 21178
iter_count: 21179
iter_count: 21180
iter_count: 21181
iter_count: 21182
iter_count: 21183
iter_count: 21184
iter_count: 21185
iter_count: 21186
iter_count: 21187
iter_count: 21188
iter_count: 21189
iter_count: 21190
iter_count: 21191
iter_count: 21192
iter_count: 21193
iter_count: 21194
iter_count: 21195
iter_count: 21196
iter_count: 21197
iter_count: 21198
iter_count: 21199
iter_count: 21200
iter_count: 21201
iter_count: 21202
iter_count: 21203
iter_count: 21204
iter_count: 21205
iter_count: 21206
iter_count: 21207
iter_count: 21208
iter_count: 21209
iter_count: 21210
iter_count: 21211
iter_count: 21212
iter_count

iter_count: 21614
iter_count: 21615
iter_count: 21616
iter_count: 21617
iter_count: 21618
iter_count: 21619
iter_count: 21620
iter_count: 21621
iter_count: 21622
iter_count: 21623
iter_count: 21624
iter_count: 21625
iter_count: 21626
iter_count: 21627
iter_count: 21628
iter_count: 21629
iter_count: 21630
iter_count: 21631
iter_count: 21632
iter_count: 21633
iter_count: 21634
iter_count: 21635
iter_count: 21636
iter_count: 21637
iter_count: 21638
iter_count: 21639
iter_count: 21640
iter_count: 21641
iter_count: 21642
iter_count: 21643
iter_count: 21644
iter_count: 21645
iter_count: 21646
iter_count: 21647
iter_count: 21648
iter_count: 21649
iter_count: 21650
iter_count: 21651
iter_count: 21652
iter_count: 21653
iter_count: 21654
iter_count: 21655
iter_count: 21656
iter_count: 21657
iter_count: 21658
iter_count: 21659
iter_count: 21660
iter_count: 21661
iter_count: 21662
iter_count: 21663
iter_count: 21664
iter_count: 21665
iter_count: 21666
iter_count: 21667
iter_count: 21668
iter_count